In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd gdrive/My\ Drive


/content/gdrive/My Drive


In [6]:
cd Job\ Drive\ 

/content/gdrive/My Drive/Job Drive 


In [7]:
cd Saathi/

/content/gdrive/My Drive/Job Drive /Saathi


In [15]:
# Import Libraries
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, LSTM, Concatenate, Reshape, GRU, Bidirectional
from keras import optimizers
from keras.models import load_model
import json, argparse, os
import re
import io
import sys
from keras.models import Model


In [182]:
## Set some global variables
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 100
LSTM_DIM = 100
DROPOUT = 0.2
NUM_CLASSES = 4
LEARNING_RATE = 0.001
NUM_EPOCHES = 10
BATCH_SIZE = 200

In [163]:
# Read training data
train = pd.read_csv('data/train.txt', sep='\t')
train.head()

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [164]:
# Read test data
test = pd.read_csv('data/test.txt', sep='\t')
test.head()

,id,turn1,turn2,turn3,label
0,0,Hmm,What does your bio mean?,I don’t have any bio,others
1,1,What you like,very little things,Ok,others
2,2,Yes,How so?,I want to fuck babu,others
3,3,what did you guess,what what,fuck,others
4,4,We ?,of course we will!,What gender movies you like??,others


In [ ]:
print (train.shape)
print (test.shape)

In [165]:
# Each class value count of traingin data
train.label.value_counts(normalize=True)*100

others    49.562334
angry     18.255968
sad       18.113395
happy     14.068302
Name: label, dtype: float64

In [166]:
# Each class value count of test data
test.label.value_counts(normalize=True)*100

others    84.897441
angry      5.409330
happy      5.155201
sad        4.538029
Name: label, dtype: float64

In [167]:
# Sample training data
train.head()

,id,turn1,turn2,turn3,label
0,0,Don't worry I'm girl,hmm how do I know if you are,What's ur name?,others
1,1,When did I?,saw many times i think -_-,No. I never saw you,angry
2,2,By,by Google Chrome,Where you live,others
3,3,U r ridiculous,I might be ridiculous but I am telling the truth.,U little disgusting whore,angry
4,4,Just for time pass,wt do u do 4 a living then,Maybe,others


In [168]:
# Remove ?? !!! ..... duplicate special character
def remove_duplicate(line):
  '''
    Remove repeated special char
  '''
  repeatedChars = ['.', '?', '!', ',']
  for c in repeatedChars:
      lineSplit = line.split(c)
      while True:
          try:
              lineSplit.remove('')
          except:
              break
      cSpace = ' ' + c + ' '    
      line = cSpace.join(lineSplit)
  
  line = line.strip().split('\t')
  return line

In [169]:
## Remove spaces
def cleanSpec(x):
  duplicateSpacePattern = re.compile(r'\ +')
  return duplicateSpacePattern.sub(' ', x)

In [170]:
## Preprocess training data
train['turn1'] = train['turn1'].apply(lambda x: x.lower())
train['turn2'] = train['turn2'].apply(lambda x: x.lower())
train['turn3'] = train['turn3'].apply(lambda x: x.lower())

train['turn1'] = train['turn1'].apply(lambda x: remove_duplicate(x))
train['turn2'] = train['turn2'].apply(lambda x: remove_duplicate(x))
train['turn3'] = train['turn3'].apply(lambda x: remove_duplicate(x))

train['turn1'] = train['turn1'].apply(lambda x: ' <eos> '.join(x))
train['turn2'] = train['turn2'].apply(lambda x: ' <eos> '.join(x))
train['turn3'] = train['turn3'].apply(lambda x: ' <eos> '.join(x))

train['turn1'] = train['turn1'].apply(lambda x: cleanSpec(x))
train['turn2'] = train['turn2'].apply(lambda x: cleanSpec(x))
train['turn3'] = train['turn3'].apply(lambda x: cleanSpec(x))

In [171]:
# Preprocess test data
test['turn1'] = test['turn1'].apply(lambda x: x.lower())
test['turn2'] = test['turn2'].apply(lambda x: x.lower())
test['turn3'] = test['turn3'].apply(lambda x: x.lower())

test['turn1'] = test['turn1'].apply(lambda x: remove_duplicate(x))
test['turn2'] = test['turn2'].apply(lambda x: remove_duplicate(x))
test['turn3'] = test['turn3'].apply(lambda x: remove_duplicate(x))

test['turn1'] = test['turn1'].apply(lambda x: ' <eos> '.join(x))
test['turn2'] = test['turn2'].apply(lambda x: ' <eos> '.join(x))
test['turn3'] = test['turn3'].apply(lambda x: ' <eos> '.join(x))

test['turn1'] = test['turn1'].apply(lambda x: cleanSpec(x))
test['turn2'] = test['turn2'].apply(lambda x: cleanSpec(x))
test['turn3'] = test['turn3'].apply(lambda x: cleanSpec(x))

In [172]:
## Tokenization training data and train tokenizer
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.turn1+train.turn2+train.turn3)


In [190]:
## Tokenization 
u1_tokens = tokenizer.texts_to_sequences(train.turn1)
u2_tokens = tokenizer.texts_to_sequences(train.turn2)
u3_tokens = tokenizer.texts_to_sequences(train.turn3)
u1_test_tokens = tokenizer.texts_to_sequences(test.turn1)
u2_test_tokens = tokenizer.texts_to_sequences(test.turn2)
u3_test_tokens = tokenizer.texts_to_sequences(test.turn3)


In [174]:
#!unzip ../../W2VecVectors/glove.840B.300d.zip 

In [175]:
## https://github.com/declare-lab/conv-emotion/tree/master/bc-LSTM
def getEmbeddingMatrix(wordIndex):
    """Populate an embedding matrix using a word-index. If the word "happy" has an index 19,
       the 19th row in the embedding matrix should contain the embedding vector for the word "happy".
    Input:
        wordIndex : A dictionary of (word : index) pairs, extracted using a tokeniser
    Output:
        embeddingMatrix : A matrix where every row has 100 dimensional GloVe embedding
    """
    embeddingsIndex = {}
    # Load the embedding vectors from ther GloVe file
    with io.open(os.path.join('./', 'glove.840B.300d.txt'), encoding="utf8") as f:
        for line in f:
            values = line.split(' ')
           # print(values)
            word = values[0]
            embeddingVector = np.array([float(val) for val in values[1:]])
            embeddingsIndex[word] = embeddingVector
    
    print('Found %s word vectors.' % len(embeddingsIndex))
    
    # Minimum word index of any word is 1. 
    embeddingMatrix = np.zeros((len(wordIndex) + 1, EMBEDDING_DIM))
    for word, i in wordIndex.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            # words not found in embedding index will be all-zeros.
            embeddingMatrix[i] = embeddingVector
    
    return embeddingMatrix

In [68]:
wordIndex = tokenizer.word_index
embeddingMatrix = getEmbeddingMatrix(wordIndex)

Found 2196016 word vectors.


In [154]:
embeddingMatrix.shape

(47989, 300)

In [191]:
u1_tokens = pad_sequences(u1_tokens, maxlen=MAX_SEQUENCE_LENGTH)
u2_tokens = pad_sequences(u2_tokens, maxlen=MAX_SEQUENCE_LENGTH)
u3_tokens = pad_sequences(u3_tokens, maxlen=MAX_SEQUENCE_LENGTH)
u1_test_tokens = pad_sequences(u1_test_tokens, maxlen=MAX_SEQUENCE_LENGTH)
u2_test_tokens = pad_sequences(u2_test_tokens, maxlen=MAX_SEQUENCE_LENGTH)
u3_test_tokens = pad_sequences(u3_test_tokens, maxlen=MAX_SEQUENCE_LENGTH)


In [177]:
emotion2label = {"others":0, "happy":1, "sad":2, "angry":3}
train.label.replace(emotion2label, inplace=True)
test.label.replace(emotion2label, inplace=True)

In [178]:
train_labels = to_categorical(np.array(train.label))
test_labels = to_categorical(np.array(test.label))


In [179]:
train_labels

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.]], dtype=float32)

In [183]:
## Create a model
x1 = Input(shape=(100,), dtype='int32', name='main_input1')
x2 = Input(shape=(100,), dtype='int32', name='main_input2')
x3 = Input(shape=(100,), dtype='int32', name='main_input3')
embeddingLayer = Embedding(embeddingMatrix.shape[0],
                                EMBEDDING_DIM,
                                weights=[embeddingMatrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=False)
emb1 = embeddingLayer(x1)
emb2 = embeddingLayer(x2)
emb3 = embeddingLayer(x3)

lstm = Bidirectional(LSTM(LSTM_DIM, dropout=DROPOUT))
lstm1 = lstm(emb1)
lstm2 = lstm(emb2)
lstm3 = lstm(emb3)
inp = Concatenate(axis=-1)([lstm1, lstm2, lstm3])
inp = Reshape((3, 2*LSTM_DIM, )) (inp)
lstm_up = LSTM(LSTM_DIM, dropout=DROPOUT)
out = lstm_up(inp)
out = Dense(NUM_CLASSES, activation='softmax')(out)
adam = optimizers.adam(lr=LEARNING_RATE)
model = Model([x1,x2,x3],out)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['acc'])
print(model.summary())

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input1 (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
main_input2 (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
main_input3 (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_10 (Embedding)        (None, 100, 300)     14396700    main_input1[0][0]                
                                                                 main_input2[0][0]          

In [184]:
model.fit([u1_tokens,u2_tokens, u3_tokens], train_labels, epochs = NUM_EPOCHES, batch_size = BATCH_SIZE)

Epoch 1/10
30160/30160 [==============================] - 552s 18ms/step - loss: 0.9830 - acc: 0.5974
Epoch 2/10
30160/30160 [==============================] - 533s 18ms/step - loss: 0.6817 - acc: 0.7417
Epoch 3/10
30160/30160 [==============================] - 548s 18ms/step - loss: 0.5811 - acc: 0.7799
Epoch 4/10
30160/30160 [==============================] - 534s 18ms/step - loss: 0.5235 - acc: 0.8057
Epoch 5/10
30160/30160 [==============================] - 546s 18ms/step - loss: 0.4737 - acc: 0.8240
Epoch 6/10
30160/30160 [==============================] - 546s 18ms/step - loss: 0.4403 - acc: 0.8385
Epoch 7/10
30160/30160 [==============================] - 525s 17ms/step - loss: 0.4157 - acc: 0.8487
Epoch 8/10
30160/30160 [==============================] - 525s 17ms/step - loss: 0.3967 - acc: 0.8562
Epoch 9/10
30160/30160 [==============================] - 541s 18ms/step - loss: 0.3745 - acc: 0.8631
Epoch 10/10
30160/30160 [==============================] - 529s 18ms/step - loss: 

In [192]:
predictions = model.predict([u1_test_tokens, u2_test_tokens, u3_test_tokens], batch_size=BATCH_SIZE)
predictions = predictions.argmax(axis=1)

In [201]:
model.save('ERCModel.h5')

In [200]:
from sklearn.metrics import confusion_matrix, classification_report


print(classification_report(test_labels.argmax(axis=1), predictions))

              precision    recall  f1-score   support

           0       0.95      0.92      0.94      4677
           1       0.59      0.54      0.57       284
           2       0.60      0.67      0.63       250
           3       0.57      0.83      0.67       298

    accuracy                           0.89      5509
   macro avg       0.68      0.74      0.70      5509
weighted avg       0.90      0.89      0.89      5509



**Conclusion**


*   I got 89% accuracy on the test data using deep learning LSTM model

